<a href="https://colab.research.google.com/github/Phillistan16/imoco_recon/blob/colab_demo/colab-imoco-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iterative Motion Compensation (iMoCo) Reconstruction Demo
This CoLab notebook is a demo of the iMoCo method
https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.27998
https://github.com/PulmonaryMRI/imoco_recon


# Setup

<!-- By default, this colab notebook should have a GPU runtime environment. To check this, go to **Runtime > Change runtime type** and see if **GPU** is selected as **Hardware accelerator**. -->

To setup, we will install [SigPy](http://sigpy.readthedocs.io), ANTs, and clone the iMoCo github repo.

In [1]:
# install dependent packages sigpy, h5py, antspy
!pip install sigpy h5py
!pip install antspyx
! git clone https://github.com/PulmonaryMRI/imoco_recon

# Need to install ANTs, or convert to ANTsPy

% cd imoco_recon/imoco_py


     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 254.6MB 57kB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
Cloning into 'imoco_recon'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 126 (delta 40), reused 108 (delta 30), pack-reused 0
Receiving objects: 100% (126/126), 59.17 KiB | 338.00 KiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/imoco_recon/imoco_py


# Download and Load dataset

Now, let's download and load a sample Lung UTE dataset. The dataset is hosted on Zenodo: https://zenodo.org/record/3733776#.XoJlNC2ZPOQ.

<!-- The `download_lung_dataset()` function will download the datasets if it cannot find them under `data/lung/`. It takes around 8 minutes to download the datasets. -->